# The Battle of Neighbourhoods (part 2)

### Problem definition

This Notebook examines and analyses different neighbourhoods in a particular area with regards to the number/different types of venues and crimes. Besides, it investigates whether exist any correlation between the numbers/types of venues crimes committed during a specific period.

#### Importing necessary Libraries

In [8]:
# Import necessary Libraries

import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import requests # library to handle requests

import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Folium installed
Libraries imported.


# Data acquisition

In [9]:
pip install xlrd

     |████████████████████████████████| 112kB 6.9MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Data source

In this study, I examine and analyse different wards in a particular county in the UK, Surrey County.  The data set which includes information related to the numbers and categories of crime that have been committed in each borough and ward can be found on [Surrey-i website](https://www.surreyi.gov.uk/) via this [link](https://www.surreyi.gov.uk/dataset?q=Master%20Crime%20Category%20by%20Ward%20). The data set has included Geocode of each ward. However, due to licence requirement, I was unable to use this information. So, I had to examine other possibilities to achieve the required geographical coordinates. Wards are the administrative division of a borough containing different areas or neighbourhoods. I couldn't find any dataset or specific website with this required information where be able to extract the data systematically. So, I had to choose a postcode in the centre of each ward manually.  Later on, I merge this information with the dataset containing all available postcodes in the UK and their latitude and longitude coordinates where I can estimate the latitude and longitude of the centre of each ward. 

I provide the list and links to the datasets that I have used in this study in the following:
1. Master Crime Category by Ward.xls   available at: https://www.surreyi.gov.uk/dataset?q=Master%20Crime%20Category%20by%20Ward%20
2. Wards by postcode.xls available at: https://github.com/SepidehN/Coursera_Capstone/blob/master/Wards_by_postcode.xls
3. UkPostcodes.csv   available at: https://www.freemaptools.com/download-uk-postcode-lat-lng.htm


### Downloading and reading Data into a DataFrame

In [ ]:
#!wget -O Master_Crime_Category_by_Ward.xls https://www.surreyi.gov.uk/download/surrey-crime-by-category-ward/83d9ce81-d251-46a1-ad36-79e70320aeb2/Master%20Crime%20Category%20by%20Ward.xls
#df = pd.read_excel('Master_Crime_Category_by_Ward.xls')


#### or

In the following, I read data containing crime records by their categories for all Wadrs in surrey county from the specified link.

In [10]:
file = 'https://www.surreyi.gov.uk/download/surrey-crime-by-category-ward/83d9ce81-d251-46a1-ad36-79e70320aeb2/Master%20Crime%20Category%20by%20Ward.xls'
df_ward_crimes = pd.read_excel(file)
# take a look at the dataset
df_ward_crimes.head()


,Month,Geocode,Borough,Ward,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES
0,2018-04-01,E10000030,Surrey total,Surrey total,36,301,437,489,5692,1738,131,1224,722,202,15,87,178
1,2018-04-01,E07000207,Elmbridge,Borough total,3,54,51,49,638,195,13,143,73,15,1,12,22
2,2018-04-01,E07000208,Epsom & Ewell,Borough total,4,18,76,35,477,130,8,90,46,16,1,31,10
3,2018-04-01,E07000209,Guildford,Borough total,4,36,22,64,709,206,17,207,78,27,4,1,20
4,2018-04-01,E07000210,Mole Valley,Borough total,4,20,31,36,357,105,6,54,48,21,1,7,14


Checking the size of DataFrame

In [11]:
df_ward_crimes.shape

(3924, 17)

In the following, I read data containing all Wadrs, their postcode and Geographical coordinates.

In [12]:
file = 'https://raw.githubusercontent.com/SepidehN/Coursera_Capstone/master/SurreyWardsByPostcodeAndCoordinates.csv'
df_Wards_by_Postcodes_Coordinates = pd.read_csv(file)
# take a look at the dataset
df_Wards_by_Postcodes_Coordinates.head()

,PostCode,Ward,Latitude,Longitude
0,KT10 0JH,Claygate,51.360700,-0.34010
1,KT12 4AR,Hersham South,51.360400,-0.41550
2,KT24 5SJ,Effingham,51.271200,-0.41140
3,KT24 6EJ,Clandon and Horsley,51.257400,-0.45580
4,CR3 0BP,Whyteleafe,53.757656,-2.48656


In [13]:
df_Wards_by_Postcodes_Coordinates.shape  #Checking the size of DataFrame

(206, 4)

In [14]:
df_Wards_by_Postcodes_Coordinates.tail(10)  #Checking the DataFrame

,PostCode,Ward,Latitude,Longitude
196,TW18 2FA,Staines,51.429713,-0.499212
197,TW19 7UH,Stanwell North,51.452996,-0.481883
198,TW20 0RF,Englefield Green East,51.434061,-0.530115
199,TW20 0UA,Englefield Green West,51.434061,-0.530115
200,TW20 9LJ,Egham Town,51.434061,-0.530115
201,TW17 9HG,Shepperton Town,51.391500,-0.448300
202,GU21 3AS,Goldsworth West,51.320726,-0.583653
203,GU21 2HA,Hermitage and Knaphill South,51.314199,-0.609309
204,NaN,Kingfield and Westfield,NaN,NaN
205,NaN,Mayford and Sutton Green,NaN,NaN


# Data preparation

#### Cleaning

The df_ward_crimes DataFrame contains rows which are not needed in our study, so I drop them.

In [15]:
rslt_df = df_ward_crimes.loc[lambda df_ward_crimes: (df_ward_crimes['Ward'] != 'Surrey total') & (df_ward_crimes['Ward'] != 'Borough total')] 
rslt_df.head()

,Month,Geocode,Borough,Ward,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES
12,2018-04-01,E05007251,EL,Claygate,0,1,0,4,26,12,1,4,3,0,0,0,0
13,2018-04-01,E05007253,EL,Cobham Fairmile,1,2,2,2,26,10,0,4,3,0,0,1,0
14,2018-04-01,E05007252,EL,Cobham and Downside,0,0,1,1,43,9,0,24,3,3,1,0,1
15,2018-04-01,E05007254,EL,Esher,2,5,2,4,28,7,0,4,4,0,0,0,0
16,2018-04-01,E05007255,EL,Hersham North,0,8,0,2,37,14,0,5,6,0,0,0,2


In [16]:
rslt_df.shape

(3708, 17)

In our data, the name of Boroughs have been provided by their abbreviation; for clarity, I replace them with their full name using a dictionary

In [17]:
Boro_dic={'EL':'Elmbridge','EP':'Epsom & Ewell','GD':'Guildford','MV':'Mole Valley', 'RB':'Reigate & Banstead','RM':'Runnymede','SL':'Spelthorne', 'SH':'Surrey Heath','TD':'Tandridge','WV':'Waverley','WK':'Woking'}
rslt_df.replace( to_replace=Boro_dic, value=None, inplace=True) # give full name to Boroughs utilising above dictinary    
rslt_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4278: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,Month,Geocode,Borough,Ward,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES
12,2018-04-01,E05007251,Elmbridge,Claygate,0,1,0,4,26,12,1,4,3,0,0,0,0
13,2018-04-01,E05007253,Elmbridge,Cobham Fairmile,1,2,2,2,26,10,0,4,3,0,0,1,0
14,2018-04-01,E05007252,Elmbridge,Cobham and Downside,0,0,1,1,43,9,0,24,3,3,1,0,1
15,2018-04-01,E05007254,Elmbridge,Esher,2,5,2,4,28,7,0,4,4,0,0,0,0
16,2018-04-01,E05007255,Elmbridge,Hersham North,0,8,0,2,37,14,0,5,6,0,0,0,2


Remove columns (i.e., 'Month') which is unnecessary our analysis.  

In [18]:
rslt_df.drop(columns=['Month'], inplace=True)  #remove unrequired column
rslt_df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Geocode,Borough,Ward,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES
12,E05007251,Elmbridge,Claygate,0,1,0,4,26,12,1,4,3,0,0,0,0
13,E05007253,Elmbridge,Cobham Fairmile,1,2,2,2,26,10,0,4,3,0,0,1,0
14,E05007252,Elmbridge,Cobham and Downside,0,0,1,1,43,9,0,24,3,3,1,0,1
15,E05007254,Elmbridge,Esher,2,5,2,4,28,7,0,4,4,0,0,0,0
16,E05007255,Elmbridge,Hersham North,0,8,0,2,37,14,0,5,6,0,0,0,2


##### Check for null values

In [19]:
rslt_df.isnull().values.any()

False

In [20]:
df_Wards_by_Postcodes_Coordinates.isnull().values.any()

True

#### I've decided to drop all rows with null values

In [21]:
df_Wards_by_Postcodes_Coordinates.dropna(inplace=True)   #drop all rows which has null values

In [22]:
df_Wards_by_Postcodes_Coordinates.isnull().values.any()

False

In [23]:
df_Wards_by_Postcodes_Coordinates.shape

(204, 4)

To conclude how many crime have been committed in each Ward, I used 'groupby' method

In [24]:
rslt_group=rslt_df.groupby(['Ward', 'Borough'],as_index=False).sum()   #grouping crime data based on wards 
rslt_group.head()

,Ward,Borough,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES
0,Addlestone Bourneside,Runnymede,0,27,29,43,495,190,21,85,54,8,0,0,20
1,Addlestone North,Runnymede,3,47,63,63,932,300,16,244,112,14,0,11,29
2,"Alfold, Cranleigh Rural and Ellens Green",Waverley,0,7,11,15,176,57,15,29,35,1,0,0,0
3,Ash South and Tongham,Guildford,3,40,23,33,562,175,6,176,63,9,0,1,15
4,Ash Vale,Guildford,1,24,10,22,225,76,2,32,41,1,1,3,8


#### Merging two dataFrames into one

To have required information (Ward and borough name, crime history, coordinates) in one DataFrame, I've used 'merge' with two parameters: on='Ward', how='inner'

In [25]:
df_total=pd.merge(rslt_group, df_Wards_by_Postcodes_Coordinates, on='Ward', how='inner')
df_total.head()

,Ward,Borough,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES,PostCode,Latitude,Longitude
0,Addlestone Bourneside,Runnymede,0,27,29,43,495,190,21,85,54,8,0,0,20,KT15 2SJ,51.370651,-0.490785
1,Addlestone North,Runnymede,3,47,63,63,932,300,16,244,112,14,0,11,29,KT15 2TN,51.376479,-0.496456
2,"Alfold, Cranleigh Rural and Ellens Green",Waverley,0,7,11,15,176,57,15,29,35,1,0,0,0,GU6 8JX,51.139195,-0.523246
3,Ash South and Tongham,Guildford,3,40,23,33,562,175,6,176,63,9,0,1,15,gu10 1da,51.238075,-0.730088
4,Ash Vale,Guildford,1,24,10,22,225,76,2,32,41,1,1,3,8,GU12 5ND,51.271993,-0.727461


In [26]:
df_total.shape

(204, 18)

#### Visualising data

To check all coordinates and wards on the map, I've used Folium package. The following function, get an address and return latitude and longitude. This coordinate can be used on folium.Map to Visualise and create the map associated with that location.

##### Create a map of Surrey with wards superimposed on top

In [27]:
def get_location(address):
    """Get the address and return geographical cordinations of the address.i.e., latitude and longitude."""
    geolocator = Nominatim(user_agent="t_explorer")
    location = geolocator.geocode(address)
    if location:
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude = float('nan')
        longitude = float('nan')
            
    return latitude,longitude

In [28]:
# create map of Surrey using latitude and longitude values 
address='Surrey, UK'
[lat,lon] = get_location(address)
map_surrey = folium.Map(location=[lat,lon], zoom_start=10) 

Using the map created above, the following code, add a marker with label for each Ward in surrey on the map.

In [29]:
# add markers to map
for lat, lon, boro, ward in zip(df_total['Latitude'],df_total['Longitude'], df_total['Borough'], df_total['Ward']):
    label = '{},{}'.format(boro, ward)
   # print(label)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
           [lat,lon],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=True).add_to(map_surrey)
map_surrey  

## Utilise the Foursquare API to explore boroughs and wards

The Foursquare API provides location based information about venues, users, photos and so on. To use this API, I need to set up the following variables. Note that you need to have an account on developer.foursquare.com, where you can find your own values for the following variables.

In [30]:
#Define Foursquare Credentials and Version
CLIENT_ID = 'EOZDYROV2XFKJM5MGI10GCO5V5CRUEHSSR4YQKFM0U2132WU' #  Foursquare ID
CLIENT_SECRET = 'LTBESQL2W3GB5R5LYK4QHAXHIEYZAI2KILAN0A4EYKNLBNGK' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### As an example, I explore Onslow ward in Guildford Borough

To access the data related 'Onslow ward', I need to retrieve its index

In [31]:
inx = int(df_total[df_total['Ward']=='Onslow'].index[0])   # retrive the index
inx

137

In [32]:
#retrive name and location of the first interst
ward_name = df_total.loc[inx,'Ward']
ward_latitude = df_total.loc[inx,'Latitude']
ward_longitude = df_total.loc[inx,'Longitude']

Here is where I actually use Foursquare to get required venues information. I need to create the requested URL as follow:

In [33]:
# Creat the url to create the GET request URL
url= ' https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION, ward_latitude, ward_longitude,2000,100)
results=requests.get(url).json()

In [128]:
#results

The following function extracts the category of the venue

In [34]:
def get_category_type(row):
    """extracts the category of the venue."""
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

The following code creates a DataFrame containing name, category and coordinates of each venue around the centre of Onslow ward.

In [35]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]  

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

,name,categories,lat,lng
0,The Drummond,Pub,51.241429,-0.576112
1,Worplesdon Road Cafe,Café,51.249920,-0.588563
2,Waitrose & Partners,Supermarket,51.238376,-0.573900
3,AirHop Trampoline Park,Indoor Play Area,51.245293,-0.584993
4,Spectrum Ice Rink,Skating Rink,51.249102,-0.562761
5,The King's Head,Pub,51.242587,-0.571006
6,Craggy Island,Climbing Gym,51.260114,-0.568006
7,Fresco Delikatessen,Café,51.235247,-0.575967
8,Red Rose,Indian Restaurant,51.253017,-0.570879
9,Meat The Greek,Greek Restaurant,51.234819,-0.573284


In [36]:
df_total.loc[inx]  # reviewing data crime related to onslow ward.

Ward                          Onslow
Borough                    Guildford
ROBBERY                            1
DOMESTIC_BURGLARY                 24
VEHICLE_CRIME                     31
VIOLENCE_WITH_INJURY              45
TNO                              654
VIOLENCE_WITHOUT_INJURY          166
NON_DOMESTIC_BURGLARY              7
THEFT_HANDLING                   273
CRIMINAL_DAMAGE                   49
DRUG_OFFENCES                     22
FRAUD_FORGERY                      1
VEHICLE_INTERFERENCE               2
OTHER_OFFENCES                    15
PostCode                     GU2 7SR
Latitude                     51.2495
Longitude                  -0.585628
Name: 137, dtype: object

## Explore all wards in surrey's boroughs

Now, it is time to explore all venues around all wards in Surrey. So I need to replicate what I did in onslow ward case, to all the wards

The following function retrieve all venues information associated with all wards in Surrey and return them as a DataFrame

In [37]:
def getNearbyVenues(wards, latitudes, longitudes, radius=2000, LIMIT=500):
    """ get nearby venues for all neighbourhoods in Surrey"""
    
    venues_list = []
    for wrd, lat, lng in zip(wards, latitudes, longitudes):
        #print(name)
        # creat url and #  the API request
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            wrd,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Ward', 
                  'Wards Latitude', 
                  'Wards Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)


In [38]:
#run above function on each ward and create a new dataframe called Surrey_wards_venues
Surrey_wards_venues = getNearbyVenues(wards = df_total['Ward'],
                                      latitudes = df_total['Latitude'],
                                   longitudes = df_total['Longitude']
                                  )

Surrey_wards_venues

,Ward,Wards Latitude,Wards Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addlestone Bourneside,51.370651,-0.490785,Bread & Roses,51.371284,-0.489750,Breakfast Spot
1,Addlestone Bourneside,51.370651,-0.490785,Waitrose & Partners,51.370806,-0.491607,Supermarket
2,Addlestone Bourneside,51.370651,-0.490785,Mystica,51.372081,-0.485512,Indian Restaurant
3,Addlestone Bourneside,51.370651,-0.490785,Weybridge Garden Centre,51.364206,-0.484804,Garden Center
4,Addlestone Bourneside,51.370651,-0.490785,Topgolf,51.359576,-0.490412,Golf Driving Range
...,...,...,...,...,...,...,...
7274,Worplesdon,51.283824,-0.597393,Asperion Hillside Hotel,51.277159,-0.609353,Hotel
7275,Worplesdon,51.283824,-0.597393,Worplesdon Railway Station (WPL),51.288142,-0.582391,Train Station
7276,Worplesdon,51.283824,-0.597393,Sutton Green Garden Centre,51.281717,-0.574000,Garden Center
7277,Worplesdon,51.283824,-0.597393,Worplesdon Golf Club,51.296752,-0.609818,Golf Course


##### Numbers of venues per each ward

Since, I am interested in number of venues in each ward, I've used 'groupby' method on 'ward' and count() to calculate the total number of venues in each Ward.

In [39]:
Surrey_wards_venues_grp=Surrey_wards_venues.groupby('Ward', as_index=False).count()
print(Surrey_wards_venues_grp.shape)
Surrey_wards_venues_grp.head()


(204, 7)


,Ward,Wards Latitude,Wards Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Addlestone Bourneside,35,35,35,35,35,35
1,Addlestone North,60,60,60,60,60,60
2,"Alfold, Cranleigh Rural and Ellens Green",5,5,5,5,5,5
3,Ash South and Tongham,28,28,28,28,28,28
4,Ash Vale,28,28,28,28,28,28


To merge information that I acquired utilising FourSquare with Crime record information of wards, I've used the following: 

In [40]:
# creat DataFrame comtaing number of venues and crime records for all wards in Surrey
df_crime_venues_count=pd.merge(Surrey_wards_venues_grp, df_total, on='Ward', how='inner')
df_crime_venues_count

,Ward,Wards Latitude,Wards Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Borough,ROBBERY,DOMESTIC_BURGLARY,...,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES,PostCode,Latitude,Longitude
0,Addlestone Bourneside,35,35,35,35,35,35,Runnymede,0,27,...,21,85,54,8,0,0,20,KT15 2SJ,51.370651,-0.490785
1,Addlestone North,60,60,60,60,60,60,Runnymede,3,47,...,16,244,112,14,0,11,29,KT15 2TN,51.376479,-0.496456
2,"Alfold, Cranleigh Rural and Ellens Green",5,5,5,5,5,5,Waverley,0,7,...,15,29,35,1,0,0,0,GU6 8JX,51.139195,-0.523246
3,Ash South and Tongham,28,28,28,28,28,28,Guildford,3,40,...,6,176,63,9,0,1,15,gu10 1da,51.238075,-0.730088
4,Ash Vale,28,28,28,28,28,28,Guildford,1,24,...,2,32,41,1,1,3,8,GU12 5ND,51.271993,-0.727461
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Witley and Hambledon,5,5,5,5,5,5,Waverley,0,48,...,10,26,32,1,0,10,2,GU8 4DG,51.155723,-0.612949
200,Woldingham,50,50,50,50,50,50,Tandridge,0,25,...,0,18,7,1,0,0,2,CR3 7JH,53.757656,-2.486560
201,Woodcote,12,12,12,12,12,12,Epsom & Ewell,2,28,...,7,111,40,17,0,5,5,KT18 6HY,51.304840,-0.261511
202,Woodham,22,22,22,22,22,22,Runnymede,0,35,...,8,34,32,2,1,2,14,KT15 3SA,51.345525,-0.513134


# Methodology

I use the k-mean clustering algorithm, a simple but powerful unsupervised machine learning technique. k-mean learns from the properties of data and divides it into different (hopefully optimal) groups.
Many clustering algorithms are available. But perhaps the most widely used one is k-means available in sklearn.cluster.KMeans. One of the main drawbacks of the k-mean algorithm is the difficulty of predicting the number of suitable clusters. To overcome this, I ran the algorithm with different parameter k and analysed the outcome of each cluster and finally, I have chosen the best k that can divide the wards more explicitly. 



In [41]:
#remove unnecessarily columns
df_crime_venues_count.drop(['Wards Latitude','Wards Longitude','Venue Latitude','Venue Longitude', 'PostCode','Latitude','Longitude','Venue Category'] ,axis=1, inplace=True)
df_crime_venues_count.head(5)

,Ward,Venue,Borough,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES
0,Addlestone Bourneside,35,Runnymede,0,27,29,43,495,190,21,85,54,8,0,0,20
1,Addlestone North,60,Runnymede,3,47,63,63,932,300,16,244,112,14,0,11,29
2,"Alfold, Cranleigh Rural and Ellens Green",5,Waverley,0,7,11,15,176,57,15,29,35,1,0,0,0
3,Ash South and Tongham,28,Guildford,3,40,23,33,562,175,6,176,63,9,0,1,15
4,Ash Vale,28,Guildford,1,24,10,22,225,76,2,32,41,1,1,3,8


I need to normalise data before applying k-mean algorithm. So, I need to remove any non-numerical columns from the DataFrame.

In [42]:
#remove non-numerical columns
df_crime_venues_count_No_wrd_bor = df_crime_venues_count.drop(['Ward' , 'Borough'],axis=1)
df_crime_venues_count_No_wrd_bor.head()

,Venue,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES
0,35,0,27,29,43,495,190,21,85,54,8,0,0,20
1,60,3,47,63,63,932,300,16,244,112,14,0,11,29
2,5,0,7,11,15,176,57,15,29,35,1,0,0,0
3,28,3,40,23,33,562,175,6,176,63,9,0,1,15
4,28,1,24,10,22,225,76,2,32,41,1,1,3,8


In [43]:
# Import necessary Libraries for normalisation
from sklearn.preprocessing import StandardScaler

#### Normalisation

In [44]:
X = df_crime_venues_count_No_wrd_bor.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


array([[-0.79269356, -0.32728619, -0.04422278, ..., -0.61195935,
        -1.08116886,  0.43976237],
       [ 0.56620969,  1.04792253,  1.62278363, ..., -0.61195935,
         1.27889234,  1.08880073],
       [-0.79269356, -1.70249491, -0.92675558, ..., -0.61195935,
        -1.08116886, -1.00254509],
       ...,
       [ 0.11324194, -0.25852576,  0.34801402, ..., -0.61195935,
        -0.00841377, -0.64196822],
       [-0.79269356,  0.2227973 , -0.92675558, ..., -0.02030197,
        -0.65206682,  0.00707013],
       [-0.79269356,  0.70412035, -0.33840038, ...,  0.57135542,
        -0.4375158 ,  0.29553163]])

## k-mean clustering

In [45]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [46]:
num_clusters = 5 #number of clusters

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[1 3 2 1 2 1 0 1 3 0 2 2 2 0 0 3 2 2 2 1 2 1 2 2 2 2 2 1 1 3 2 2 2 0 2 3 2
 0 1 1 0 2 1 0 2 3 2 2 0 1 1 2 3 2 1 0 2 1 1 4 1 1 0 2 2 1 2 2 1 2 2 2 2 2
 2 2 3 2 4 1 2 2 1 2 1 2 1 3 2 1 1 1 1 2 1 1 1 0 2 1 4 3 2 1 2 1 0 3 0 3 1
 2 1 2 1 0 1 3 1 1 1 2 0 0 1 1 0 1 1 1 2 0 2 2 2 1 2 1 0 1 1 2 2 2 2 2 2 2
 3 3 3 2 1 0 2 2 2 2 0 1 0 2 1 2 4 1 1 3 1 0 1 3 0 1 0 1 0 1 2 2 1 2 3 1 1
 1 2 1 1 2 0 1 2 2 2 1 1 1 2 2 2 1 2 1]


In [47]:
# add label column to data
df_crime_venues_count["Labels"] = labels
df_crime_venues_count.head(5)

,Ward,Venue,Borough,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES,Labels
0,Addlestone Bourneside,35,Runnymede,0,27,29,43,495,190,21,85,54,8,0,0,20,1
1,Addlestone North,60,Runnymede,3,47,63,63,932,300,16,244,112,14,0,11,29,3
2,"Alfold, Cranleigh Rural and Ellens Green",5,Waverley,0,7,11,15,176,57,15,29,35,1,0,0,0,2
3,Ash South and Tongham,28,Guildford,3,40,23,33,562,175,6,176,63,9,0,1,15,1
4,Ash Vale,28,Guildford,1,24,10,22,225,76,2,32,41,1,1,3,8,2


In [48]:
#combine data to add wards'coordinates to the table 
merged_data=pd.merge(df_crime_venues_count, df_Wards_by_Postcodes_Coordinates, on='Ward', how='inner')

In [49]:
merged_data.head()

,Ward,Venue,Borough,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES,Labels,PostCode,Latitude,Longitude
0,Addlestone Bourneside,35,Runnymede,0,27,29,43,495,190,21,85,54,8,0,0,20,1,KT15 2SJ,51.370651,-0.490785
1,Addlestone North,60,Runnymede,3,47,63,63,932,300,16,244,112,14,0,11,29,3,KT15 2TN,51.376479,-0.496456
2,"Alfold, Cranleigh Rural and Ellens Green",5,Waverley,0,7,11,15,176,57,15,29,35,1,0,0,0,2,GU6 8JX,51.139195,-0.523246
3,Ash South and Tongham,28,Guildford,3,40,23,33,562,175,6,176,63,9,0,1,15,1,gu10 1da,51.238075,-0.730088
4,Ash Vale,28,Guildford,1,24,10,22,225,76,2,32,41,1,1,3,8,2,GU12 5ND,51.271993,-0.727461


In [50]:
#Summarise clusters data
merged_data.groupby('Labels').mean()

,Venue,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES,Latitude,Longitude
Labels,,,,,,,,,,,,,,,,
0,42.035714,1.750000,42.250000,57.357143,32.500000,473.571429,123.785714,9.500000,114.321429,50.500000,6.142857,0.892857,12.642857,10.821429,51.442748,-0.446201
1,39.152778,1.916667,33.541667,29.138889,42.791667,475.597222,149.972222,10.263889,99.236111,63.972222,10.263889,1.000000,3.888889,13.750000,51.402735,-0.541550
2,29.000000,0.493827,22.802469,14.666667,18.308642,209.444444,65.061728,5.222222,35.209877,27.283951,3.592593,0.395062,2.716049,8.000000,51.523783,-0.716930
3,33.105263,5.052632,45.736842,53.789474,104.368421,983.421053,315.263158,15.894737,231.473684,113.842105,21.894737,2.526316,7.894737,33.578947,51.325434,-0.396887
4,76.250000,8.500000,41.250000,46.500000,155.500000,1860.750000,506.000000,36.500000,723.250000,168.750000,52.750000,8.500000,6.000000,64.250000,51.313261,-0.478432


## Visualising clusters on the map

In [51]:
# a Geographical coordinates to create a map
latitude=51.2362
longitude= -0.5704

In the following, I've visualised clusters of the wards with different colors and added labels containing their name, borough and cluster number.

In [52]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
#ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.brg(np.linspace(0, 1, len(x)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, wrd, boro, cluster in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['Ward'],merged_data['Borough'], merged_data['Labels']):
    label = folium.Popup(str(boro)+', '+ str(wrd) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Result 
Cluster 4, which includes wards in the centre of two big towns in Surrey area (i.e., Guildford and Epsom), shows that more venues can cause more risk in the area. As the Table above shows the number of crime committed in this cluster are significantly high. On the other hand, Cluster 2, is associated with wards that have a very low density of venues and a slight risk in terms of crime committed. However, Cluster 3, despite having a low density of the venues, it has high risk where make these wards undesirable since they are neither enough popular nor enough safe. These wards are mostly inside the M25 or near big town such as Woking or Redhill. Cluster 1 and 0 are slightly similar. They are wards with moderate density and relatively moderate risk. However, as presented on the map, Cluster 0 includes wards where mostly are located inside the M25 band which make increases in specific categories of crime such as theft handling and vehicle interference in these areas. In general, **the model confirms we should expect crime rises in the more busy/popular areas**.

### More investigation on wards and analysing crime committed in each one

In this section, I've analyses data to Find the wards with maximum number of crime in each crime category 

In [46]:
df_total.head()

,Ward,Borough,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES,PostCode,Latitude,Longitude
0,Addlestone Bourneside,Runnymede,0,27,29,43,495,190,21,85,54,8,0,0,20,KT15 2SJ,51.370651,-0.490785
1,Addlestone North,Runnymede,3,47,63,63,932,300,16,244,112,14,0,11,29,KT15 2TN,51.376479,-0.496456
2,"Alfold, Cranleigh Rural and Ellens Green",Waverley,0,7,11,15,176,57,15,29,35,1,0,0,0,GU6 8JX,51.139195,-0.523246
3,Ash South and Tongham,Guildford,3,40,23,33,562,175,6,176,63,9,0,1,15,gu10 1da,51.238075,-0.730088
4,Ash Vale,Guildford,1,24,10,22,225,76,2,32,41,1,1,3,8,GU12 5ND,51.271993,-0.727461


In [48]:
#Dropping unnecessary columns
temp_df_total=df_total.drop(['Ward','Borough', 'PostCode','Latitude', 'Longitude','TNO'],axis=1)
temp_df_total.head()

,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES
0,0,27,29,43,190,21,85,54,8,0,0,20
1,3,47,63,63,300,16,244,112,14,0,11,29
2,0,7,11,15,57,15,29,35,1,0,0,0
3,3,40,23,33,175,6,176,63,9,0,1,15
4,1,24,10,22,76,2,32,41,1,1,3,8


In [49]:
#Finding the wards with maximum number of crime in each crime category 
wrd_inx=temp_df_total.idxmax(axis=0)    #retrive maximum value in each column
df_ward_by_most_freq_crime=pd.DataFrame(wrd_inx)  # define a datafram
df_ward_by_most_freq_crime.reset_index(level=0, inplace=True)   #or df['Crime Category'] = df.index
df_ward_by_most_freq_crime.columns=['Crime Category', 'Ward\'s index'] # add name for each column
df_ward_by_most_freq_crime

,Crime Category,Ward's index
0,ROBBERY,148
1,DOMESTIC_BURGLARY,138
2,VEHICLE_CRIME,8
3,VIOLENCE_WITH_INJURY,78
4,VIOLENCE_WITHOUT_INJURY,78
5,NON_DOMESTIC_BURGLARY,59
6,THEFT_HANDLING,100
7,CRIMINAL_DAMAGE,78
8,DRUG_OFFENCES,177
9,FRAUD_FORGERY,78


So, I need to retrieve the name of each ward via index I've obtained in previous section.

In [7]:
inx=df_ward_by_most_freq_crime['Ward\'s index']
inx                                      

NameError: name 'df_ward_by_most_freq_crime' is not defined

In [51]:
# Retrieve information via index.
disadvantaged_wards=merged_data.iloc[inx]
disadvantaged_wards

,Ward,Venue,Borough,ROBBERY,DOMESTIC_BURGLARY,VEHICLE_CRIME,VIOLENCE_WITH_INJURY,TNO,VIOLENCE_WITHOUT_INJURY,NON_DOMESTIC_BURGLARY,THEFT_HANDLING,CRIMINAL_DAMAGE,DRUG_OFFENCES,FRAUD_FORGERY,VEHICLE_INTERFERENCE,OTHER_OFFENCES,Labels,PostCode,Latitude,Longitude
148,Redhill East,30,Reigate & Banstead,14,46,71,105,1301,370,20,423,128,32,1,7,37,4,RH1 2AE,51.248010,-0.163984
138,Oxshott and Stoke D'abernon,32,Elmbridge,0,79,51,22,377,108,7,53,23,6,1,7,9,0,KT11 2SU,51.329081,-0.410664
8,Ashford North and Stanwell South,32,Spelthorne,3,71,112,110,1050,329,9,195,128,17,3,21,25,4,TW15 3AH,51.442853,-0.470161
78,Friary and St. Nicolas,99,Guildford,7,44,35,220,2390,704,32,822,217,108,12,10,102,1,GU2 4JA,51.249485,-0.585628
78,Friary and St. Nicolas,99,Guildford,7,44,35,220,2390,704,32,822,217,108,12,10,102,1,GU2 4JA,51.249485,-0.585628
59,Epsom Town,60,Epsom & Ewell,6,38,65,116,1670,470,47,679,146,22,10,6,27,1,KT17 1EE,51.337124,-0.259085
100,Holy Trinity,92,Guildford,9,42,18,136,1725,372,42,863,123,39,7,2,38,1,GU1 3AP,51.236721,-0.569804
78,Friary and St. Nicolas,99,Guildford,7,44,35,220,2390,704,32,822,217,108,12,10,102,1,GU2 4JA,51.249485,-0.585628
177,Thorpe,34,Runnymede,1,13,60,27,558,115,14,142,45,109,3,3,14,2,TW18 3NF,51.413449,-0.511450
78,Friary and St. Nicolas,99,Guildford,7,44,35,220,2390,704,32,822,217,108,12,10,102,1,GU2 4JA,51.249485,-0.585628


In [52]:
# Drop unnecessery columns
disadvantaged_wards.drop(['ROBBERY','DOMESTIC_BURGLARY','VEHICLE_CRIME','VIOLENCE_WITH_INJURY','TNO','VIOLENCE_WITHOUT_INJURY','NON_DOMESTIC_BURGLARY','THEFT_HANDLING','CRIMINAL_DAMAGE','DRUG_OFFENCES','FRAUD_FORGERY','VEHICLE_INTERFERENCE','OTHER_OFFENCES','PostCode','Latitude','Longitude'],axis=1,inplace=True)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


The following output is the list of disadvantaged wards with maximum number of committed crime. We need to add another column indicating the crime category. 

In [53]:
disadvantaged_wards

,Ward,Venue,Borough,Labels
148,Redhill East,30,Reigate & Banstead,4
138,Oxshott and Stoke D'abernon,32,Elmbridge,0
8,Ashford North and Stanwell South,32,Spelthorne,4
78,Friary and St. Nicolas,99,Guildford,1
78,Friary and St. Nicolas,99,Guildford,1
59,Epsom Town,60,Epsom & Ewell,1
100,Holy Trinity,92,Guildford,1
78,Friary and St. Nicolas,99,Guildford,1
177,Thorpe,34,Runnymede,2
78,Friary and St. Nicolas,99,Guildford,1


The following, shows the list of disadvantaged wards, their boroughs, number of venues, cluster label and the crime categories which have the highest record among all Wards. 

In [54]:
temp_catg=df_ward_by_most_freq_crime['Crime Category']
disadvantaged_wards['Crime Category']=list(temp_catg)
disadvantaged_wards

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Ward,Venue,Borough,Labels,Crime Category
148,Redhill East,30,Reigate & Banstead,4,ROBBERY
138,Oxshott and Stoke D'abernon,32,Elmbridge,0,DOMESTIC_BURGLARY
8,Ashford North and Stanwell South,32,Spelthorne,4,VEHICLE_CRIME
78,Friary and St. Nicolas,99,Guildford,1,VIOLENCE_WITH_INJURY
78,Friary and St. Nicolas,99,Guildford,1,VIOLENCE_WITHOUT_INJURY
59,Epsom Town,60,Epsom & Ewell,1,NON_DOMESTIC_BURGLARY
100,Holy Trinity,92,Guildford,1,THEFT_HANDLING
78,Friary and St. Nicolas,99,Guildford,1,CRIMINAL_DAMAGE
177,Thorpe,34,Runnymede,2,DRUG_OFFENCES
78,Friary and St. Nicolas,99,Guildford,1,FRAUD_FORGERY
